### Implementing simple neural network from scratch
In this notebook I implemented a simple neural network without using advanced ML libraries.


In [ ]:
import pandas as pd
import numpy as np
from keras.utils import np_utils
import matplotlib.pyplot as plt
from common import *

In [ ]:
class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    def forward_pass(self, input):
        raise NotImplementedError

    def backpropagation(self, output_error, learning_rate):
        raise NotImplementedError

class FullyConnectedLayer(Layer):
    def __init__(self, input_size, output_size, init_method='random'):
        # TODO: #1 init_method
        # weiths and bias are initialized randomly
        self.weights = np.random.rand(input_size, output_size) - 0.5
        self.bias = np.random.rand(1, output_size) - 0.5

    def forward_pass(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output

    def backpropagation(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)

        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error

class ActivationComponent(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime

    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output

    def backward_propagation(self, output_error, learning_rate):
        return self.activation_prime(self.input) * output_error

class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None
        self.convergence=[]

    # add layer to network
    def add(self, layer):
        self.layers.append(layer)

    # set loss to use
    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

    def predict(self, input_data):
        samples = len(input_data)
        result = []
        for i in range(samples):
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)

        return result

    def fit(self, x_train, y_train, epochs, learning_rate):
        samples = len(x_train)
        for i in range(epochs):
            err = 0
            for j in range(samples):
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_propagation(output)
                err += self.loss(y_train[j], output)

                error = self.loss_prime(y_train[j], output)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)
                    
            err /= samples
            self.convergence.append(err)
            print('epoch %d/%d   error=%f' % (i+1, epochs, err))

Data visualization
- Iris-setosa : red
- Iris-versicolor: green
- Iris-virginica: blue

In [ ]:
#get iris
iris = pd.read_csv('iris.csv')
plot_iris_data(iris)

In [ ]:
net = Network()
net.add(FullyConnectedLayer(4, 100))
net.add(ActivationComponent(tanh, tanh_prime))
net.add(FullyConnectedLayer(100, 50))
net.add(ActivationComponent(tanh, tanh_prime))
net.add(FullyConnectedLayer(50, 10))
net.add(ActivationComponent(tanh, tanh_prime))
net.add(FullyConnectedLayer(10, 3))

# convert species to one-hot encoding
iris['Species'] = iris['Species'].map({'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2})

# split data into training and test set (80%/20%)
iris_train = iris.sample(frac=0.8)
iris_test = iris.drop(iris_train.index)
# convert to numpy arrays
iris_train = iris_train.to_numpy()
iris_test = iris_test.to_numpy()
# normalize data (0-1)
for i in range(4):
    iris_train[:, i] = (iris_train[:, i] - iris_train[:, i].min()) / (iris_train[:, i].max() - iris_train[:, i].min())
    iris_test[:, i] = (iris_test[:, i] - iris_test[:, i].min()) / (iris_test[:, i].max() - iris_test[:, i].min())
iris_train_x = iris_train[:, :4]
iris_train_y = np_utils.to_categorical(iris_train[:, 4])
iris_test_x = iris_test[:, :4]
iris_test_y =  iris_test[:, 4]
iris_train_x =iris_train_x.reshape(iris_train_x.shape[0], 1, 4)

net.use(mse, mse_prime)
net.fit(iris_train_x,iris_train_y, epochs=10, learning_rate=0.1)

#make accuracy prediction
pred = net.predict(iris_test_x)
print(iris_test_y)
print(pred)
for prediction in pred:
    print(np.argmax(prediction))
# calculate accuracy
correct = 0
for i in range(len(iris_test_y)):
    if np.argmax(pred[i]) == iris_test_y[i]:
        correct += 1
print('accuracy: %f' % (correct / len(iris_test_y)))
# plot convergence
plt.plot(range(len(net.convergence)), net.convergence)
plt.show()